In [3]:
import scipy.io
data = scipy.io.loadmat('data/url.mat')

**create list for labels and data, where one entry is the data for the day with this index**

In [4]:
import numpy as np

num_of_days = 120
X, Y = [], []

for i in range(num_of_days):
    day_data = data["Day" + str(i)]
    X.append(day_data[0][0][0])
    Y.append(day_data[0][0][1])

**flatten y to not be a nested array**

In [5]:
# y-data are lists of numpy-arrays

for i in range(len(Y)):
    Y[i] = [element for sublist in Y[i] for element in sublist]
   
# y-data becomes a list of lists

In [25]:
from scipy.sparse import csc_matrix

def concatenate_csc_matrices_by_columns(matrix1, matrix2):
    new_data = np.concatenate((matrix1.data, matrix2.data))
    
    
    new_indices = np.concatenate((matrix1.indices, matrix2.indices))
    
    new_ind_ptr = matrix2.indptr + len(matrix1.data)
    new_ind_ptr = new_ind_ptr[1:]
    new_ind_ptr = np.concatenate((matrix1.indptr, new_ind_ptr))

    return csc_matrix((new_data, new_indices, new_ind_ptr))


x = concatenate_csc_matrices_by_columns(X[1], X[2])
print(x.shape)

(20000, 6463922)


### Combine data and value vectors of days 0- 16

In [30]:
from numpy import vstack

print(X[0].shape)
print(X[1].shape)
x2 = vstack((X[1], X[2]))
            
print(x2.shape)
print(x2)

## So what now:
# try to stack the data 
#(find other way, redo the function aboce to fit my needs, workaround for the error from scipy.vstack)
# try to do fit with array like [X[1], X[2], X[3] ..], [Y[1], Y[2], Y[3] ..], 

##print(type(Y[0]))

(16000, 3231961)
(20000, 3231961)
(2, 1)
[[<20000x3231961 sparse matrix of type '<class 'numpy.float64'>'
	with 2304202 stored elements in Compressed Sparse Column format>]
 [<20000x3231961 sparse matrix of type '<class 'numpy.float64'>'
	with 2300129 stored elements in Compressed Sparse Column format>]]


### Train once on data for days 0-16

In [6]:
from sklearn.svm import LinearSVC

clf = LinearSVC()

for i in range(17):
    clf.fit(x2, Y[i])

AttributeError: 'LinearSVC' object has no attribute 'partial_fit'

### Evaluate for all days

In [ ]:
from sklearn.metrics import accuracy_score

Y_test_preds = []

for i in range(0, num_of_days):
    Y_preds = clf.predict(X[i])
    print("Error-rate Day {}   : {}".format(i,1 - accuracy_score(Y[i], Y_preds)))